# Analyzing Hubway Trips in SQL

In [1]:
import sql

In [3]:
%reload_ext sql
%sql sqlite:///hubway.db
%config SqlMagic.autocommit=False

**Please write your queries below in the empty boxes same as you write in a SQL Window**

Just append **%sql** before your query like this '%sql Select * from Trips limit 5;'

In [4]:
%sql Select * from Trips limit 2

 * sqlite:///hubway.db
(sqlite3.OperationalError) no such table: Trips
[SQL: Select * from Trips limit 2]
(Background on this error at: http://sqlalche.me/e/e3q8)


**About the Dataset and Schema**

The database has two tables, trips and stations. To begin with, we'll look at the trips table. It contains the following columns:


**Trips:**

id — A unique integer that serves as a reference for each trip

duration — The duration of the trip, measured in seconds

start_date — The date and time the trip began

start_station — An integer that corresponds to the id column in the stations table for the station the trip started at

end_date — The date and time the trip ended

end_station — The 'id' of the station the trip ended at

bike_number — Hubway's unique identifier for the bike used on the trip

sub_type — The subscription type of the user. "Registered" for users with a membership, "Casual" for users without a membership

zip_code — The zip code of the user (only available for registered members)

birth_date — The birth year of the user (only available for registered members)

gender — The gender of the user (only available for registered members)


**Station:**


id — A unique identifier for each station (corresponds to the start_station and end_station columns in the trips table)

station — The station name

municipality — The municipality that the station is in (Boston, Brookline, Cambridge or Somerville)

lat — The latitude of the station

lng — The longitude of the station

In [4]:
%sql select * from Stations limit 3

 * sqlite:///hubway.db
Done.


id,station,municipality,lat,lng
3,Colleges of the Fenway,Boston,42.340021,-71.100812
4,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616
5,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179


## Analytical Questions Test in SQL

Q1. What was the duration of the longest trip? What was the average trip duration? What was the smallest trip duration?

###### A1. Duration of the longest trip is "9999 sec", 
######       Minimum trip duration is "0 sec", 
######       Average trip duration is "912.41 sec"

In [20]:
%sql SELECT *, MAX(duration) FROM Trips; 

 * sqlite:///hubway.db
Done.


id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender,MAX(duration)
541247,9999,2012-09-26 18:34:00,54,2012-09-26 21:21:00,54,T01078,Casual,,,,9999


In [14]:
%sql SELECT *, MIN(duration) FROM Trips; 

 * sqlite:///hubway.db
Done.


id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender,MIN(duration)
11586,0,2011-08-09 14:31:00,44,2011-08-09 14:31:00,44,B00430,Registered,'02446,1956.0,Male,0


In [15]:
%sql SELECT *, AVG(duration) FROM Trips; 

 * sqlite:///hubway.db
Done.


id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender,AVG(duration)
1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male,912.4096819046612


Q2. How many trips were taken by 'Registered' users?

Hint: Use Sub Type to Filter

###### A2. "110,5192" trips were taken by 'Registered' users

In [24]:
%sql SELECT COUNT(*) Regitered_users FROM Trips WHERE sub_type = 'Registered'; 

 * sqlite:///hubway.db
Done.


Regitered_users
1105192


Q3. How many trips were taken by male users in comparison to Female users? for Registered users only

Hint: Group By Gender and Calculate Count

###### A3. Male registered users taken 833858 trips , while female registerd users taken 271334 trips 

In [28]:
%sql SELECT gender, COUNT(*) Regitered_users FROM Trips WHERE sub_type = 'Registered' GROUP BY gender;

 * sqlite:///hubway.db
Done.


gender,Regitered_users
Female,271333
Female,1
Male,833858


Q4. Do registered or casual users take longer trips? 

Hint: Group By Sub Type and Calculate Average

###### A4. Casual users taken longer trips in comparison to registerd users

In [32]:
%sql SELECT sub_type, AVG(duration) AVG_trip_duration FROM Trips GROUP BY sub_type;

 * sqlite:///hubway.db
Done.


sub_type,AVG_trip_duration
Casual,1519.6438967403815
Registered,657.0260669639302


Q5. Which bike was used for the most trips? 

Hint : Use Bike Number and Limit Results to 1

###### A5. BIke with Bike_number  "b00225" was used for most of the trips

In [2]:
%sql SELECT bike_number, COUNT(*) trips FROM Trips \
GROUP BY bike_number \
ORDER BY 2 DESC LIMIT 1;
#####3 order by count (*)

UsageError: Line magic function `%sql` not found.


Q6. What is the average duration of trips by users over the age of 30? 

Hint : DoB is already in years and use Airthmetic operator to caluculate Age like (2020-DOB)

###### A6. Average duration of trips by users over the age of 30 is  "917 sec" 

In [24]:
%sql SELECT AVG(duration) FROM trips \
WHERE (2020-birth_date) > 30

 * sqlite:///hubway.db
Done.


avg(duration)
916.5142035958438


Q7. Which stations are most frequently used for round trips? 

Hint: You will require to use Join Station Table, Also a Round trip is considered as where Start and Stop Stations are same. Limit to top 5 Stations

###### A7.  stations with id 135, 9, 66, 85, 20 were most frequently used for round trips

In [33]:
%sql select s.station, count(*) number_trips
from trips t
join stations as s on t.start_station = s.id
WHERE t.start_station = t.end_station \
group by t.start_station
order by 2 desc
limit 5;

 * sqlite:///hubway.db
Done.


station,id,start_station,end_station
359 Broadway - Broadway at Fayette Street,135,135,135
Agganis Arena - 925 Comm Ave.,9,9,9
Allston Green District - Commonwealth Ave & Griggs St,66,66,66
Andrew Station - Dorchester Ave at Humboldt Pl,85,85,85
Aquarium Station - 200 Atlantic Ave.,20,20,20


Q8. How many trips start and end in different municipalities?

Hint: We need to JOIN the trips table to the stations table twice. Once ON the start_station column and then ON the end_station

###### A8. "1496904" trips start and end in different municipalities

In [3]:
%sql %sql SELECT COUNT(*) FROM trips t \
JOIN stations s ON t.start_station = s.id \
JOIN stations st ON t.end_station = st.id \
WHERE s.start_municipality <> s.end_station

 * sqlite:///hubway.db
Done.


count(t.id)
1496904


Q9. How many trips incurred additional fees (lasted longer than 30 minutes)?

Hint: Use an Arithmetic function to calculate Duration in Minutes and Use in Where Clause

###### A9.  "119917" trips incurred additional fees

In [5]:
%sql SELECT COUNT(t.id) FROM trips t \
WHERE (duration / 60) > 30

 * sqlite:///hubway.db
Done.


COUNT(t.id)
119917


Q10. Which bike was used for the longest total time? Provide the answer in Hours

Hint: Use an Group by Bike Number to sum Duration in Seconds and Limit Result to 1

###### A10. Bike with bike_number  "B00490" was used for longest total time of  "571 hours"

In [21]:
%sql SELECT bike_number, SUM(duration) longest_total_time FROM trips \
GROUP BY bike_number \
ORDER BY longest_total_time DESC \
LIMIT 1

 * sqlite:///hubway.db
Done.


bike_number,longest_total_time
B00490,2058643


In [9]:
%sql SELECT bike_number, SUM(duration)/3600 longest_total_time FROM trips \
GROUP BY bike_number \
ORDER BY longest_total_time DESC \
# order by 2
LIMIT 1

 * sqlite:///hubway.db
Done.


bike_number,longest_total_time
B00490,571


Q11. Did registered or casual users take more round trips?

Hint: Recall Q7 and Modify your query to answer by Sub Type

###### A11. Casual users take more round trips

In [15]:
%sql SELECT sub_type, AVG(duration), SUM(duration) FROM stations \
JOIN trips ON stations.id = trips.start_station \
WHER trips.start_station = trips.end_station \
GROUP BY sub_type 

 * sqlite:///hubway.db
Done.


sub_type,avg(duration),sum(duration)
Casual,3058.2311294566343,126693341
Registered,867.3554923344398,27438791


Q12. Which municipality had the most frequent Station End?

Hint: Join the Station table on End Station and then Group by Municipality to find answer

###### A12. "Boston" had the most frequent station end

In [25]:
%sql SELECT count(*) trips, s.municipality, end_station FROM trips t \
JOIN stations s ON t.end_station = s.id \
GROUP BY s.municipality \
order by trips DESC
#order by 2 desc
limit 1

 * sqlite:///hubway.db
Done.


trips,municipality,end_station
1212364,Boston,23
297011,Cambridge,73
39921,Somerville,78
20677,Brookline,69
